In [12]:
#  Ebnable HTML/CSS 
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")


___
Enter Team Member Names here (double click to edit):

- Name 1: Ryan Shuhart
- Name 2: Alex Matsunami
- Name 3: Cory Nichols



________
<a id="top"></a>
# Live Session Assignment Two
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (.ipynb file, remember to save it!!) or HTML of the rendered notebook before the end of class.

## Contents
* <a href="#Loading">Loading the Classification Data</a>
* <a href="#using_trees">Using Decision Trees - Gini</a>
* <a href="#entropy">Using Decision Trees - Entropy</a>
* <a href="#multi">Multi-way Splits</a>
* <a href="#sklearn">Decision Trees in Scikit-Learn</a>

________________________________________________________________________________________________________
<a id="Loading"></a>
<a href="#top">Back to Top</a>
## Loading the Classification Data
Please run the following code to read in the "digits" dataset from sklearn's data loading module. This is identical to the first in class assignment for loading the data into matrices. `ds.data` is a matrix of feature values and `ds.target` is a column vector of the class output (in our case, the hand written digit we want to classify). Each class is a number (0 through 9) that we want to classify as one of ten hand written digits. 



In [13]:
from sklearn.datasets import load_digits
import numpy as np
from __future__ import print_function

ds = load_digits()

# this holds the continuous feature data
print('features shape:', ds.data.shape) # there are 1797 instances and 64 features per instance
print('target shape:', ds.target.shape )
print('range of target:', np.min(ds.target),np.max(ds.target))

features shape: (1797, 64)
target shape: (1797,)
range of target: 0 9


________________________________________________________________________________________________________
<a id="using_trees"></a>
<a href="#top">Back to Top</a>
## Using Decision Trees
In the videos, we talked about the splitting conditions for different attributes. Specifically, we discussed the number of ways in which it is possible to split a node, depending on the attribute types. To understand the possible splits, we need to understand the attributes. For the question below, you might find the description in the `ds['DESCR']` field to be useful. You can see the field using `print(ds['DESCR'])`

**Question 1:** For the digits dataset, what are the type(s) of the attributes? How many attributes are there? What do they represent?


In [78]:
## Enter your comments here
## Enter your comments here
# sklearn splits based on binary or multiway
print(ds['DESCR'])

## Enter comments here
# There are 5620 instances with 64 attributes or features representing an 8x8 image of integer pixels in the range 0..16
# There are no missing attributes
# The attributes themselves are continuous and represent handwritten digits from a preprinted form
# 
print('max gini %.3f'%(1-1/float(len(np.unique(ds.target)))))

## Enter comments here

Optical Recognition of Handwritten Digits Data Set

Notes
-----
Data Set Characteristics:
    :Number of Instances: 5620
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
http://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each block. This generates
an input matrix of 8x8 where each element is a

___
## Using the gini coefficient
We talked about the gini index in the videos.  The gini coefficient for a **given split** is given by:
$$Gini=\sum_{t=1}^T \frac{n_t}{N}gini(t)$$
where $T$ is the total number of splits (2 for binary attributes), $n_t$ is the number of instances in node $t$ after splitting, and $N$ is the total number of instances in the parent node. $gini(t)$ is the **gini index for each individual node that is created by the split** and is given by:
$$gini(t)=1-\sum_{j=0}^{C-1} p(j|t)^2$$
where $C$ is the total number of possible classes and $p(j|t)$ is the probability of class $j$ in node $t$ (i.e., $n_j==$ the count of instances belonging to class $j$ in node $t$, normalized by the total number of instances in node $t$).
$$ p(j|t) = \frac{n_j}{n_t}$$ 

For the given dataset, $gini(t)$ has been programmed for you in the function `gini_index`. 

* `def gini_index(classes_in_split):`
 * To use the function, pass in a `numpy` array of the class labels for a node as (i.e., pass in the rows from `ds.target` that make up a node in the tree) and the gini will be returned for that node. 


In [79]:
# compute the gini of several examples for the starting dataset
# This function "gini_index" is written for you. Once you run this block, you 
#   will have access to the function for the notebook. You do not need to know 
#   how this function works--only what it returns 
# This function returns the gini index for an array of classes in a node.
def gini_index(classes_in_split):
    # pay no attention to this code in the function-- it just computes the gini for a given split 
    classes_in_split = np.reshape(classes_in_split,(len(classes_in_split),-1))
    unique_classes = np.unique(classes_in_split)
    gini = 1
    for c in unique_classes:
        gini -= (np.sum(classes_in_split==c) / float(len(classes_in_split)))**2
        
    return gini

In the example below, the function is used calculate the gini for splitting the dataset on feature 28, with value 2.5. In this example, we need to create two separate tree nodes: the first node has all the `ds.target` labels when feature 28 is greater than 2.5, the second node has all the rows from `ds.target` where feature 28 is less than 2.5. The steps are outlined below. **Read this carefully to understand what the code does below in the block following this.**
- Feature 28 is saved into a separate variable `feature28 = ds.data[:,28]`
- First all the target classes for the first node are calculated using `numpy` indexing `ds.target[feature28>2.5]` 
 - Note: this grabs all the rows in `ds.target` (the classes) which have feature 28 greater than 2.5 (similar to indexing in pandas)
- Second, those classes are passed into the function to get the gini for the right node in this split (i.e., feature 28 being greater than the threshold 2.5). 
 - `gini_r = gini_index(ds.target[feature28>2.5])`
- Third, the gini is calculated for the left node in the tree. This grabs only the rows in `ds.target` where feature 28 is less than 2.5.
     - `gini_l = gini_index(ds.target[feature28<=2.5])`
- Combining the gini indices is left as an exercise in the next section

In [80]:
#==========================Use the gini_index Example===============
# get the value for this feature as a column vector 
# (this is like grabbing one column of the record table)
feature28 = ds.data[:,28]

# if we split on the value of 2.5, then this is the gini for each resulting node:
gini_r = gini_index(ds.target[feature28>2.5]) # just like in pandas, we are sending in the rows where feature28>2.5
gini_l = gini_index(ds.target[feature28<=2.5]) # and sending the rows where feature28<=2.5

# compute gini example. This splits on attribute '28' with a value of 2.5
print('gini for right node of split:', gini_r)
print('gini for left node of split:', gini_l)

gini for right node of split: 0.884585786767
gini for left node of split: 0.711540756654


**Question 2:** Now, using the above values `gini_r` and `gini_l`. Calculate the combined Gini for the entire split. You will need to write the weighted summation (based upon the number of instances inside each node). To count the number of instances greater than a value using numpy, you can use broadcasting, which is a special way of indexing into a numpy array. For example, the code `some_array>5` will return a new numpy array of true/false elements. It is the same size as `some_array` and is marked true where the array is greater than `5`, and false otherwise. By taking the `sum` of this array, we can count how many times `some_array` is greater than `5`. 

`counts = sum(some_array>5)` 
 
You will need to use this syntax to count the values in each node as a result of splitting.  

In [81]:
## Enter your code here

countR = sum(feature28>2.5) # count up the instances of feature28 that are > 2.5
countL = sum(feature28<=2.5) # '' <= 2.5
totCount = float(len(feature28))

# sum weighted avg of parent node
print(countR, countL)
print('The total gini of the split for a threshold of 2.5 is: %.3f'%(countR/totCount*gini_r+countL/totCount*gini_l))
# countR makes up the majority of the split, pulling the combined closer to .88 vs .71 for the left node (<2.5)
print('class makeup of right node:', len(np.unique(ds.target[feature28>2.5])))
print('class makeup of left node:', len(np.unique(ds.target[feature28<=2.5])))
# the right split (>2.5) still contains all classes
print('distribution of classes for right node:',
      [np.sum(ds.target[feature28>2.5]==i) for i in np.unique(ds.target[feature28>2.5])])
print('distribution of classes for left node:',
      [np.sum(ds.target[feature28<=2.5]==i) for i in np.unique(ds.target[feature28<=2.5])])
# no wonder our gini is so high...


## Enter your code here
print('The total gini of the split for a threshold of 2.5 is:','??')

1398 399
The total gini of the split for a threshold of 2.5 is: 0.846
class makeup of right node: 10
class makeup of left node: 10
distribution of classes for right node: [3, 180, 171, 180, 118, 145, 82, 169, 171, 179]
distribution of classes for left node: [175, 2, 6, 3, 63, 37, 99, 10, 3, 1]
The total gini of the split for a threshold of 2.5 is: ??


___
## Start of Live Session Coding

**Question 3:** Now we want to know which is a better split:
- `feature28` split on a value of `2.5`  
- `feature28` split on a value of `10`.  

Enter your code to find the total gini of splitting on the threshold of 10 and compare it to the total gini of splitting on threshold of 2.5 (for feature 28 only). According to gini, which threshold is better for spliting on feature 28, `threshold=2.5` or `threshold=10.0`?

In [82]:
# Enter your code here

gini_r = gini_index(ds.target[feature28>10.0])
gini_l = gini_index(ds.target[feature28<=10.0]) 

countR = sum(feature28>10.0) 
countL = sum(feature28<=10.0) 
totCount = float(len(feature28))

print('The total gini of the split for a threshold of 10 is: %.3f'%(countR/totCount*gini_r+countL/totCount*gini_l))
print('This is not better than the split on 2.5, we would split on 2.5 over 10')

The total gini of the split for a threshold of 10 is: 0.864
This is not better than the split on 2.5, we would split on 2.5 over 10


___
<a id="entropy"></a>
<a href="#top">Back to Top</a>
## Entropy based splitting
We discussed entropy as well in the video as another means of splitting. We calculated entropy for a node $t$ by:
$$ Entropy(t) = -\sum p(j|t) \log p(j|t) $$
where $p(j|t)$ is the same as above. To combine Entropy measures from a set of nodes, t = {1,...,T} we use: 
$$Entropy_{split}=\sum_{t=1}^T \frac{n_t}{N}Entropy(t)$$ 
where $n_t$ and $N$ are the same as defined above for the $Gini$. Information gain is calculated by subtracting the Entropy of the split from the Entropy of the parent node before splitting:
$$InfoGain = Entropy(p)-Entropy_{split}$$
where $p$ is the parent node before splitting. You are given an equation for calculating the $Entropy(t)$ of  node $t$. It works exactly like the `gini_index` function above, but is named `entropy_value` and returns the entropy for a node. You simply send in an array of the feature values for the node you want to calculate the entropy value for. 


In [83]:
def entropy_value(classes_in_split):
    # pay no attention to this code -- it just computes the gini for a given split 
    classes_in_split = np.reshape(classes_in_split,(len(classes_in_split),-1))
    unique_classes = np.unique(classes_in_split)
    ent = 0
    for c in unique_classes:
        p = (np.sum(classes_in_split==c) / float(len(classes_in_split)))
        ent += p * np.log(p)
        
    return -ent

In [84]:
ent_r = entropy_value(ds.target[feature28>2.5])
ent_l = entropy_value(ds.target[feature28<=2.5])

# compute entropy example. This splits on attribute '28' with a value of 2.5
print('entropy for right node of split:', ent_r)
print('entropy for left node of split:', ent_l)

entropy for right node of split: 2.18369753782
entropy for left node of split: 1.48988814128


___
**Question 4:** Calculate the **information gain** of the split when the threshold is 2.5 on `feature28`. What is the value of the information gain?

In [85]:
# Enter your code here

parent = entropy_value(ds.target)

countR = sum(feature28 > 2.5) 
countL = sum(feature28 <= 2.5)

print('The information gain of the split for a threshold of 2.5 is: %.3f'%(
        parent - (countR/totCount*ent_r+countL/totCount*ent_l)))

The information gain of the split for a threshold of 2.5 is: 0.273


**Question 5:** What is the information gain if the threshold is 10.0 on `feature28`? According to information gain, is it better to split on a threshold of 2.5 or 10? Does entropy give the same decision as gini for this example?

In [86]:
# Enter your code here

parent = entropy_value(ds.target)

ent_r = entropy_value(ds.target[feature28 > 10])
ent_l = entropy_value(ds.target[feature28 <= 10])

countR = sum(feature28 > 10.0) 
countL = sum(feature28 <= 10.0)


print('The information gain of the split for a threshold of 10 is: %.3f'%(
        parent - (countR/totCount*ent_r+countL/totCount*ent_l)))

print('This is not better than the split on 2.5')
print('This is the same as gini from a result perspective, we\'d choose 2.5 to split on over 10')

The information gain of the split for a threshold of 10 is: 0.210
This is not better than the split on 2.5
This is the same as gini from a result perspective, we'd choose 2.5 to split on over 10


___
<a id="multi"></a>
<a href="#top">Back to Top</a>
## Information gain and multi-way splitting
Now assume that we can use not just a binary split, but a three way split. 

**Question 6** What is the information gain if we split feature28 on two thesholds (three separate nodes corresponding to three branches from one node) 
- node left: `feature28<2.5`, 
- node middle: `2.5<=feature28<10`, and 
- node right: `10<=feature28`? 

Is the information gain better? 

***Note***: You can index into a `numpy` array for the middle node with the following notation: `some_array[(2.5<=feature28) & (feature28<10.0)]`

In [87]:
# Enter your code here, repeating just for effect

parent = entropy_value(ds.target)

count_l = sum(feature28 < 2.5) 
count_c = sum((feature28 >= 2.5) & (feature28 < 10))
count_r = sum(feature28 >= 10)

ent_l = entropy_value(ds.target[feature28 < 2.5])
ent_c = entropy_value(ds.target[(feature28 >= 2.5) & (feature28 < 10)]) # save this for future use
ent_r = entropy_value(ds.target[feature28 >= 10])

print('The information gain of the split for a three way split is: %.3f'%(
        parent - (count_r/totCount*ent_r+count_l/totCount*ent_l+count_c/totCount*ent_c)))

The information gain of the split for a three way split is: 0.317


**Question 7**: Should we normalize the quantity that we just calculated if we want to compare it to the information gain of a binary split? Why or Why not?

##### Enter your comments here

Yes, because the information for a three way split will always be greater than the information gain for a binary split because of the number of splits and randomness from three buckets instead of two. Penalize more complicated splits with the gain ratio that takes into account the number of splits.


##### Enter your comments here

___
<a id="sklearn"></a>
<a href="#top">Back to Top</a>
## Decision Trees in scikit-learn
Scikit-learn also has an implementation of decision trees. Its available here:
- http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

**Question 8**: What algorithm does scikit-learn use for creating decision trees (i.e., ID3, C4.5, C5.0, CART, MARS, CHAID, etc.)? 

In [88]:
# Enter your answer here

# scikit-learn uses an optimised version of the CART algorithm - 
# http://scikit-learn.org/stable/modules/tree.html#tree-algorithms

# Enter your answer here

___
**Question 9**: Using the documentation, use scikit-learn to train a decision tree on the digits data. Calculate the accuracy on the training data. What is the accuracy? Did you expect the decision tree to have this kind of accuracy? Why or Why not?

In [101]:
# use scikit learn to train a decision tree
from sklearn.tree import DecisionTreeClassifier as dtree

# enter your code below here to train and predict on the same data

treefit = dtree(criterion='entropy', max_depth = 3, random_state=0)
treefit.fit(ds.data, ds.target)

# enter your code above here

from sklearn.metrics import accuracy_score 

# enter your code below here to calculate accuracy

print('accuracy of depth 3:', accuracy_score(treefit.predict(ds.data), ds.target))
print('''A dataset with 10 levels, 64 attributes, using a tree with a max depth of 3, we expected poor classification 
         accuracy. We will need to extend the depth of the decision tree while accounting for overfitting. 
         Based on the inherent variability of the data, for instance, someone can write the number 4 multiple ways, 
         we\'d expect one decision tree at a depth of 3 to not be a very good classifier
      ''')

# to illustrate
treefit = dtree(criterion='entropy', max_depth = 4, random_state=0)
treefit.fit(ds.data, ds.target)
print('accuracy of depth 4:', accuracy_score(treefit.predict(ds.data), ds.target))

# enter your code above here

accuracy of depth 3: 0.551474680022
A dataset with 10 levels, 64 attributes, using a tree with a max depth of 3, we expected poor classification 
         accuracy. We will need to extend the depth of the decision tree while accounting for overfitting. 
         Based on the inherent variability of the data, for instance, someone can write the number 4 multiple ways, 
         we'd expect one decision tree at a depth of 3 to not be a very good classifier
      
accuracy of depth 4: 0.717863105175


___
**Question 10**: Look at the other input parameters to the function `DecisionTreeClassifier` could any of them be used to help prevent the decision tree from overlearning on the data? 

Which variables might we use to control overfitting and how (explain why it might help to stop overfitting)?

In [102]:
# Enter your answer here

# Max depth can help us from overfitting by preventing the tree from creating too many leaf nodes and decision points
# that fit the data perfectly but do not generalize well. 
# we can also use max leaf nodes and minimum sample split to fight overfitting our data. For instance, we could ensure
# that a candidate split contain at least 10 observations before it's split again or we could contain the amount of 
# leafs directly in the model with max leaf nodes


# Enter your answer here

________________________________________________________________________________________________________

That's all! Please **upload your rendered notebook** and please include **team member names** in the notebook submission. Also please remember to save the notebook before uploading.